In [1]:
#Scrapping Start

from modules.scrapper_logic1 import  Scrapper1
import os
from modules.null_dates import *
from modules.dummy_func import *
from modules.combine_cdf import *
from modules.calparam import *
import cdflib
from modules.plotter import Plotting
import matplotlib.pyplot as plt
import concurrent.futures
from modules.nan_repair import *


In [7]:
'''
For scarpping H1 data by year,month,day
'''

# Scrapper1 -> year,month,start-day,end-day


year = '1998'
task = Scrapper1.get_list(int(year),'0',0,0)
main_path = f'download//{year}'

if os.path.exists(main_path):
    pass
else:
    os.makedirs(main_path)



Year 1998 has 363 dataset


In [ ]:

#scrapper call
Scrapper1.run(year,task,main_path)

# Code -> Dummy logic

In [ ]:
'''
For creating dummy cdf if data is not available for any date 
'''

dummy_path = f'{main_path}'

#path,year, st_month, ed_month
start_month = 1
end_month = 12
dummy_dates = get_null_dates(dummy_path,int(year), start_month, end_month)

print(dummy_dates)



In [ ]:

'''
write dummy files and save it ot main dataset folder according to Year
'''

for i in dummy_dates:
    yr =int(i[:4])
    
    month =int(i[4:6])
    day = int(i[6:])

    get_dummy(day,month,yr,dummy_path)
    

# Combine Logic

In [8]:
#combine


'''
Combining the dataset according to year provided
'''


lisdir = os.listdir(f'{main_path}')

if os.path.exists(f"{main_path}//combined//"):
    pass
else:
    os.makedirs(f"{main_path}//combined//")
    
for cdfs in range(len(lisdir)):
    
    cdf1,cdf2,cdf3 = cdflib.CDF(f'{main_path}/{lisdir[cdfs]}'),cdflib.CDF(f'{main_path}/{lisdir[cdfs+1]}'),cdflib.CDF(f'{main_path}/{lisdir[cdfs+2]}')
 
    path = f"{main_path}//combined//wi_h1_swe_{lisdir[cdfs].split("_")[3]} - {lisdir[cdfs+2].split("_")[3]}.cdf"

    result_cdf = merge_cdf(cdf1,cdf2,cdf3)
    combined_cdf = make_cdf(path,result_cdf)

    if cdfs+2 ==len(lisdir)-1:
        break
    
   

        

# Plotting IPPLOT

In [ ]:

'''
Plotting the dataset according to year provided
'''


if os.path.exists(f"{main_path}//PLOT//"):
    pass
else:
    os.makedirs(f"{main_path}//PLOT//")


plt.ioff()
def process_file(cdf_file_name):
    plt_name = cdf_file_name.split("_")[-1].split(".")[0]
    plt_ad = f'{main_path}//PLOT/'
    print(plt_name)
    cdf_file = cdflib.CDF(f'{main_path}//combined/{cdf_file_name}')

    time_range= cdflib.cdfepoch.to_datetime(cdf_file.varget('epoch'))


    

    BX,BY,BZ,b_mag = bmagnitude(cdf_file)
    p_density = thresold(cdf_file.varget('Proton_Np_moment'),1e4)
    vp = Vp_vec(cdf_file)
    temp = Temperature(cdf_file)
    p_beta = plasma_beta(p_density,b_mag,temp)
    tehta_angle,phi_angle = angels(BX,BY,BZ,b_mag)

    if True in np.isnan(cdf_file['BX']):    

        BX,BY,BZ,b_mag,p_density,vp,temp,p_beta,tehta_angle,phi_angle = repair_full(cdf_file,
                                                                            time_range,
                                                                            BX,BY,BZ,
                                                                            b_mag,
                                                                            p_density,
                                                                            vp,temp,
                                                                            p_beta,
                                                                            tehta_angle,
                                                                            phi_angle)
    

    plot  = Plotting(plt_name,plt_ad)

    plot.ipplot(time_range,b_mag,BX,BY,BZ,tehta_angle,phi_angle,vp,p_density,temp,p_beta)

lis_dir = os.listdir(f'{main_path}//combined')

# Use ThreadPoolExecutor to parallelize processing and plotting tasks

# with concurrent.futures.ThreadPoolExecutor(10) as executor:
#     executor.map(process_file, lis_dir)

for cdf_file in lis_dir:
    process_file(cdf_file)



    

# testing plot

In [1]:
import cdflib
from modules.plotter import Plotting
from modules.calparam import *
from modules.nan_repair import *
import pandas as pd
cdf_file = cdflib.CDF(r'SWE_H1_DATASET\2003\wi_h1_swe_20030102_v01.cdf')
# path=r'E:\SWE\ICME-ML'
time_range= cdflib.cdfepoch.to_datetime(cdf_file.varget('epoch'))




In [2]:


BX,BY,BZ,b_mag = bmagnitude(cdf_file)
p_density = thresold(cdf_file.varget('Proton_Np_moment'),1e4)
vp = Vp_vec(cdf_file)
temp = Temperature(cdf_file)
p_beta = plasma_beta(p_density,b_mag,temp)
tehta_angle,phi_angle = angels(BX,BY,BZ,b_mag)




In [3]:
plot  = Plotting("2003-01-02")
plot.ipplot(time_range,b_mag,BX,BY,BZ,tehta_angle,phi_angle,vp,p_density,temp,p_beta)